In [5]:
# Do all relevant imports
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.sparse as sps
import time
import scipy
import tables as tb

In [35]:
class Matrix_multiplicator(object): 
    def __init__(self):
        self.bl = 500 #this is the number of rows we calculate each loop
    
    def dot(self, m1, m2, filename, result_format = 'csr'): 
        starttime = time.time()
        
        l, m, n = m1.shape[0], m1.shape[1], m2.shape[1]

        self.f = tb.open_file(filename, 'w')
        filters = tb.Filters(complevel=5, complib='blosc')
        
        out_data = self.f.create_earray(self.f.root, 'data', tb.Float32Atom(), shape=(0,), filters=filters)
        out_indices = self.f.create_earray(self.f.root, 'indices', tb.Int64Atom(),shape=(0,), filters=filters)
        out_indptr = self.f.create_earray(self.f.root, 'indptr', tb.Int64Atom(), shape=(0,), filters=filters)
        out_indptr.append(np.array([0])) #this is needed as a first indptr
        max_indptr = 0
        print("Heading into loop. %s sec"%(time.time()-starttime))
        cp = time.time()
        for i in range(0, l, self.bl):
            if i % 5000 == 0 or i == 500: 
                print("Row %s out of %s, %s min."%(i, l, (time.time()-cp)/60))
                cp = time.time()
            res = m1[i:min(i+self.bl, l),:].dot(m2)
            out_data.append(res.data)
            indices = res.indices
            indptr = res.indptr
            out_indices.append(indices)
            out_indptr.append(max_indptr+indptr[1:])
            max_indptr += indices.shape[0]
            
        if result_format == 'csr': 
            m = sps.csr_matrix((self.f.root.data[:], self.f.root.indices[:], self.f.root.indptr[:]), shape=(l,n))
        elif result_format == 'csc': 
            m = sps.csc_matrix((self.f.root.data[:], self.f.root.indices[:], self.f.root.indptr[:]), shape=(l,n))
        
        self.f.close()
        print("Calculated m.dot() to %s. %s sec" %(filename, time.time()-starttime))
        
    def get_result(self, filename, l, n, result_format = 'csr'):
        h5 = tb.open_file(filename, 'r')
        if result_format == 'csr':
            m = sps.csr_matrix((h5.root.data[:], h5.root.indices[:], h5.root.indptr[:]), shape=(l,n))
        elif result_format == 'csc': 
            m = sps.csc_matrix((h5.root.data[:], h5.root.indices[:], h5.root.indptr[:]), shape=(l,n))
        h5.close()
        return m
m = Matrix_multiplicator()

In [36]:
a = sps.rand(50, 50).tocsr()
b = sps.rand(50, 50).tocsc()

In [37]:
print(a.nnz)

25


In [38]:
cp = time.time()
m.dot(a,b,'h5/test.h5')
c = m.get_result('h5/test.h5', 50, 50)
print(c)
print(time.time()-cp)

OSError: ``h5`` does not exist